# Importing libraries

In [1]:
import pandas as pd

In [2]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.3.1'

# Part 1 - Datapreprocessing

 - Applying transformation on the training. 
 - This is to avoid overfitting
 - To avoid high accuracy on training set and low accuracy on test set. This is called 'overfitting'.

**Transformations**

- Some zooms or rotations on images
- The technical term is called image augmentation
- Applying the transformations we get new images


## Preprocessing the training set

### ImageDatagen preprocessing code from Keras

https://keras.io/api/preprocessing/image/#imagedatagenerator-class

In [4]:
#rescale applies feature scaling (normalisation) on each pixel by dividing each pixel 
#value by 255. This is needed for the algorithm
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_set = train_datagen.flow_from_directory(
        'training_set',
        target_size=(64, 64), #This will make the training much faster
        batch_size=32,
        class_mode='binary') # This is because we are working on binary outcome


Found 8000 images belonging to 2 classes.


## Preprocessing the test set

In [5]:
#Remember we don't want to apply any transformations on test set, but we still need to 
#apply feature scaling

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')



Found 2000 images belonging to 2 classes.


# Part 2 - Building the CNN

### Initialising the CNN

In [6]:
# A convulution neural network is a sequence of layers. 
#Therefore we need to call the sequential class

In [7]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [8]:
## This means we are building a feature detector

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]))

### Step 2 - Pooling

In [10]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Adding the second layer

In [11]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu')) ## don't need the input layer as already added
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Step 3 - Flattening

In [12]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full connection

In [13]:
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

### Step 5 - Output layer

In [14]:
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid')) # This is because we just want a binary output.

# Part 3 - Training the CNN

### Compiling the CNN

In [15]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the cnn on the training and evaluating it on the test set

In [16]:
cnn.fit(x = train_set, validation_data = test_set, epochs = 25) # Keep changing the epochs and monitor accuracy

Epoch 1/25
250/250 [==============================] - 70s 281ms/step - loss: 0.6725 - accuracy: 0.5882 - val_loss: 0.6155 - val_accuracy: 0.6845
Epoch 2/25
250/250 [==============================] - 63s 253ms/step - loss: 0.6078 - accuracy: 0.6737 - val_loss: 0.5924 - val_accuracy: 0.6990
Epoch 3/25
250/250 [==============================] - 61s 243ms/step - loss: 0.5655 - accuracy: 0.7044 - val_loss: 0.6335 - val_accuracy: 0.6625
Epoch 4/25
250/250 [==============================] - 66s 263ms/step - loss: 0.5321 - accuracy: 0.7320 - val_loss: 0.5164 - val_accuracy: 0.7530
Epoch 5/25
250/250 [==============================] - 63s 252ms/step - loss: 0.5035 - accuracy: 0.7530 - val_loss: 0.5663 - val_accuracy: 0.7195
Epoch 6/25
250/250 [==============================] - 64s 255ms/step - loss: 0.4882 - accuracy: 0.7606 - val_loss: 0.5071 - val_accuracy: 0.7600
Epoch 7/25
250/250 [==============================] - 63s 251ms/step - loss: 0.4692 - accuracy: 0.7784 - val_loss: 0.5184 - val_ac

# Part 4 - Making a prediction

In [32]:
from keras.preprocessing import image

In [33]:
img_input = 'single_prediction/cat_or_dog_3.jpg'

In [34]:
# The size of the test image should be the same as the size of the training images.
# During image pre-processing we resized images to 64 x 64 using the target_size parameter

test_image = image.load_img(img_input,target_size=(64,64))

In [35]:
# In order to be accepted by the predict method we need to change the PIL image into an array

test_image = image.img_to_array(test_image)

#Remember our CNN was trained on batches of images, which is an extra dimension. 
#So our single prediction image needs that extra-dimension which corresponds to the batch.

test_image = np.expand_dims(test_image, axis=0)# this adds the batch dimension to the image array
result = cnn.predict(test_image)
train_set.class_indices

#remember the result is also inside a batch
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
    


In [36]:
print(prediction)



dog
